In [211]:
import pandas as pd
import numpy as np
import torch
import datetime
import os
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
from matplotlib import ticker 
import folium
import branca
from datetime import datetime, timedelta,date
from scipy.interpolate import make_interp_spline, BSpline
import plotly.express as px
import json, requests
import calmap

from IPython.core.interactiveshell import InteractiveShell
InteractiveShell.ast_node_interactivity = "all"
pd.set_option('display.max_columns', 99)
pd.set_option('display.max_rows', 99)
from tqdm import tqdm
import plotly.express as px
import plotly.subplots as subplots
import plotly.graph_objects as go
import seaborn as sns
import os
import matplotlib.pyplot as plt
from PIL import Image

from plotly.offline import init_notebook_mode, iplot, plot
init_notebook_mode(connected=True)

import warnings
warnings.filterwarnings('ignore')

%matplotlib inline

In [212]:
def get_benchmark(model_dir, model_name, location2name, target='1 wk ahead cum death'):
    baseline_fps = os.listdir(os.path.join(model_dir, model_name))
    baseline_fps = [item for item in baseline_fps if not (item.startswith('metadata') or item.endswith('.txt'))]
    horizon = 7 * int(target.split()[0])
    try:
        if len(baseline_fps)==0:
            return -1
        for i,_fp in enumerate(baseline_fps):
            tmp = pd.read_csv(os.path.join(model_dir, model_name, _fp))
            tmp = tmp[tmp['type']=='point'][tmp['target']==target]
            tmp['region'] = tmp.location.map(location2name)
            tmp = tmp[tmp.region!='US'][['forecast_date','target_end_date','value','region']]
            tmp['target_start_date'] = pd.to_datetime(tmp['target_end_date']) - timedelta(days=horizon-1)
            tmp['target_end_date'] = pd.to_datetime(tmp['target_end_date'])
            if i==0:
                df = tmp.copy()
            else:
                df = pd.concat([df,tmp], axis=0)

        return df.sort_values(['target_start_date','region']).reset_index(drop=True)
    except:
        return -1

def get_label(death_fp, target_start_date, horizon=7, group_name='Province_State'):
    deaths = pd.read_csv(death_fp)
    ts_features = [item for item in deaths.columns if item not in ['UID', 'iso2', 'iso3', 'code3', 'FIPS', 'Admin2', 'Province_State',
           'Country_Region', 'Lat', 'Long_', 'Combined_Key', 'Population', 'Province/State', 'Country/Region', 'Long']]

    deaths_us = deaths.groupby(group_name)[ts_features].sum().diff(axis=1).T
    deaths[ts_features] = deaths[ts_features].mask(deaths[ts_features]<0,0)
    deaths_us.index = pd.to_datetime(deaths_us.index)
    use_index = pd.date_range(start=target_start_date,freq='D',periods=horizon)
    deaths_us = deaths_us[deaths_us.index.isin(use_index)]
    deaths_us['target_start_date'] = pd.to_datetime(use_index[0])
    deaths_us['target_end_date'] = pd.to_datetime(use_index[-1])
    deaths_us['horizon'] = horizon
    deaths_us = deaths_us.set_index(['target_start_date','target_end_date','horizon']).stack().reset_index().rename({0:'label', group_name:'region'},
                                                                                          axis=1)
    deaths_us = deaths_us.groupby(['target_start_date','horizon','region'])['label'].sum().reset_index()
    deaths_us.loc[deaths_us['label']<0,'label'] = 0

    deaths_cum =  deaths.groupby(group_name)[ts_features].sum().T
    deaths_cum.index = pd.to_datetime(deaths_cum.index)
    deaths_cum = deaths_cum.loc[pd.to_datetime(target_start_date)- datetime.timedelta(days=1)].reset_index()
    deaths_cum.columns = ['region','cum_label']
    deaths_us = pd.merge(deaths_us,deaths_cum,how='left',on=['region'])
    deaths_us['cum_label'] = deaths_us['cum_label'] + deaths_us['label']
    
    return deaths_us


In [213]:
model_dir = '/home/zhgao/covid19-forecast-hub/data-processed/'
location_fp = '/home/zhgao/covid19-forecast-hub/data-locations/locations.csv'
death_fp = 'https://raw.githubusercontent.com/CSSEGISandData/COVID-19/master/csse_covid_19_data/csse_covid_19_time_series/time_series_covid19_deaths_US.csv'
confirmed_fp ='https://raw.githubusercontent.com/CSSEGISandData/COVID-19/master/csse_covid_19_data/csse_covid_19_time_series/time_series_covid19_confirmed_US.csv'
location = pd.read_csv(location_fp)
location2name = dict(zip(location['location'],location['location_name']))  

In [214]:
model_name_list = [item for item in os.listdir(model_dir) if '.' not in item]

In [215]:
model_results = dict()
for model_name in model_name_list:
    tmp = get_benchmark(model_dir,model_name,location2name)
    if isinstance(tmp,int) and tmp==-1:
        print("Empty results: {}".format(model_name))
    elif len(tmp)==0:
        print("Zero results: {}".format(model_name))
    else:
        model_results[model_name]=tmp

Zero results: Imperial-ensemble1
Zero results: CDDEP-SEIR_MCMC
Zero results: QJHong-Encounter
Empty results: ISUandPKU-vSEIdR
Zero results: Quantori-Multiagents
Zero results: PandemicCentral-USCounty
Empty results: Caltech-CS156
Zero results: Yu_Group-CLEP
Empty results: UCM_MESALab-FoGSEIR
Zero results: Imperial-ensemble2
Zero results: CMU-TimeSeries
Zero results: UMass-ExpertCrowd
Zero results: IBF-TimeSeries
Zero results: IQVIA_ACOE-STAN
Zero results: UVA-Ensemble
Zero results: UChicagoCHATTOPADHYAY-UnIT


In [246]:
model_name = 'MSRA-DeepST'
tmp = get_benchmark('/home/zhgao/COVID-Research/','CDC',location2name)
# tmp = get_benchmark('/home/zhgao/covid19-forecast-hub/data-processed/',model_name,location2name)
model_results[model_name]=tmp

In [247]:
forecast_start_date = ['2020-09-21','2020-09-28','2020-10-05','2020-10-12','2020-10-19','2020-10-26','2020-11-02','2020-11-09']
epiweek_end_dates = [pd.to_datetime(start_date) + timedelta(days=5) for start_date in forecast_start_date]
epiweek_start_dates = [pd.to_datetime(start_date) - timedelta(days=1) for start_date in forecast_start_date]
deaths = pd.read_csv(death_fp)
ts_features = [item for item in deaths.columns if item not in ['UID', 'iso2', 'iso3', 'code3', 'FIPS', 'Admin2', 'Province_State',
       'Country_Region', 'Lat', 'Long_', 'Combined_Key', 'Population', 'Province/State', 'Country/Region', 'Long']]
deaths = deaths.groupby('Province_State')[ts_features].sum()
deaths = deaths.stack().reset_index().rename({'Province_State':'region','level_1':'target_end_date',0:'label'},axis=1)
deaths['target_end_date'] = pd.to_datetime(deaths['target_end_date'])
deaths = deaths[deaths['target_end_date'].isin(epiweek_end_dates)]

In [248]:
regions = ['Alabama', 'Alaska', 'Arizona', 'Arkansas', 'California',
       'Colorado', 'Connecticut', 'Delaware', 'Florida', 'Georgia',
       'Hawaii', 'Idaho', 'Illinois', 'Indiana', 'Iowa', 'Kansas',
       'Kentucky', 'Louisiana', 'Maine', 'Maryland', 'Massachusetts',
       'Michigan', 'Minnesota', 'Mississippi', 'Missouri', 'Montana',
       'Nebraska', 'Nevada', 'New Hampshire', 'New Jersey', 'New Mexico',
       'New York', 'North Carolina', 'North Dakota', 'Ohio', 'Oklahoma',
       'Oregon', 'Pennsylvania', 'Rhode Island', 'South Carolina',
       'South Dakota', 'Tennessee', 'Texas', 'Utah', 'Vermont',
       'Virginia', 'Washington', 'West Virginia', 'Wisconsin', 'Wyoming']

score_results = dict()
score_results_all = dict()
score_tmp = dict()
for k,v in  model_results.items():
    v = v.drop_duplicates(['region','target_end_date'],keep='first')
    tmp = pd.merge(deaths,v,on=['region','target_end_date'],how='inner')
    dates = tmp['forecast_date'].unique()
    region_size = tmp['region'].unique().shape[0]
    score_tmp[k] = tmp
    tmp = tmp[tmp['region'].isin(regions)]
    if len(tmp['region'].unique()) > 30:
        score = (tmp['label'] - tmp['value']).map(abs).mean()
        score_results[k] = score
        
        score_all = tmp.groupby('target_end_date').apply(lambda x: (x['label'] - x['value']).map(abs).mean())
        score_results_all[k] = score_all
    else:
        print(k)

UChicago-CovidIL
SWC-TerminusCM
STH-3PU
UChicago-CovidIL_100
CovidActNow-SEIR_CAN
UChicago-CovidIL_60
UChicago-CovidIL_10_+
Google_Harvard-CPF
GT_CHHS-COVID19
UChicago-CovidIL_30_+
IHME-CurveFit
NotreDame-FRED
UChicago-CovidIL_80
Auquan-SEIR
UCSB-ACTS
UChicago-CovidIL_40
JHU_UNC_GAS-StatMechPool


In [251]:
pd.DataFrame(score_results_all).T

target_end_date,2020-09-26,2020-10-03,2020-10-10,2020-10-17,2020-10-24,2020-10-31,2020-11-07,2020-11-14
MIT_CritData-GBCF,28.460000,29.560000,31.000000,33.060000,31.440000,41.240000,55.300000,58.020000
CU-select,46.820000,56.900000,36.500000,55.180000,40.400000,46.480000,65.140000,87.440000
UMass-MechBayes,25.320000,21.960000,26.640000,22.060000,32.880000,31.520000,47.180000,41.600000
MSRA-DeepST,19.855294,20.231336,23.727839,17.495617,28.861260,22.658152,37.622149,36.265402
YYG-ParamSearch,57.397054,44.219745,21.950948,NaN,NaN,NaN,NaN,NaN
CEID-Walk,27.780800,28.505280,26.339300,28.703160,40.754240,33.918620,48.588160,47.642260
JHUAPL-Bucky,57.643360,52.724090,43.462990,40.033420,41.936290,59.455812,72.572024,58.906631
GT-DeepCOVID,28.929772,22.287184,25.689713,31.541994,42.353947,30.205513,43.013347,36.440411
CU-scenario_high,46.900000,56.620000,36.180000,43.980000,40.220000,46.380000,74.100000,73.260000
USC-SI_kJalpha,27.140000,29.300000,25.420000,27.300000,35.920000,37.580000,46.580000,37.740000


In [254]:
stats = pd.DataFrame(score_results_all).T.rank().sort_values('2020-10-31').fillna(99).astype(int)
stats.columns = ['Epiweek_{}'.format(str(week).split()[0]) for week in stats.columns]
stats[:10].style.background_gradient(cmap='viridis_r').set_precision(0)

,Epiweek_2020-09-26,Epiweek_2020-10-03,Epiweek_2020-10-10,Epiweek_2020-10-17,Epiweek_2020-10-24,Epiweek_2020-10-31,Epiweek_2020-11-07,Epiweek_2020-11-14
MSRA-DeepST,1,4,2,1,1,1,1,3
LNQ-ens1,4,5,3,2,10,2,2,8
GT-DeepCOVID,19,8,5,19,27,3,4,4
UA-EpiCovDA,24,12,14,13,18,4,17,14
COVIDhub-ensemble,6,2,6,5,11,5,6,11
UMass-MechBayes,9,7,9,3,6,6,8,12
OliverWyman-Navigator,3,17,11,7,9,7,3,5
MOBS-GLEAM_COVID,17,27,28,26,16,8,14,10
COVIDhub-baseline,14,13,10,10,24,9,12,17
CEID-Walk,16,18,7,11,23,10,11,18


In [210]:
dump_dict = dict()
tmp = stats[:10].reset_index()
for col in tmp.columns:
    dump_dict[col] = tmp[col].values.tolist()
SAVE_DIR = '../website/'
with open(os.path.join(SAVE_DIR, 'MSRA.Deaths.Predict.Rank.json'),'w') as f:
    f.write(json.dumps(dump_dict))

521

In [ ]:
stats = pd.DataFrame(score_results_all).T.sort_values('2020-10-31').fillna(99.0)[:10].T
maes = stats.mean()

In [ ]:
stats.columns = [item + '('+str(round(score,4))+')' for item,score in zip(stats.columns, maes.values)]
stats = stats.unstack().reset_index().rename({'level_0':'TeamName',
                                              'target_end_date':'Date',
                                              0:'MAE'},axis=1)

In [ ]:
fig = px.line(stats, x='Date', y='MAE',color='TeamName')
_ = fig.update_layout(title_text='MAE performance of top 10 teams')
fig.show()

In [25]:
regions = ['Alabama', 'Alaska', 'Arizona', 'Arkansas', 'California',
       'Colorado', 'Connecticut', 'Delaware', 'Florida', 'Georgia',
       'Hawaii', 'Idaho', 'Illinois', 'Indiana', 'Iowa', 'Kansas',
       'Kentucky', 'Louisiana', 'Maine', 'Maryland', 'Massachusetts',
       'Michigan', 'Minnesota', 'Mississippi', 'Missouri', 'Montana',
       'Nebraska', 'Nevada', 'New Hampshire', 'New Jersey', 'New Mexico',
       'New York', 'North Carolina', 'North Dakota', 'Ohio', 'Oklahoma',
       'Oregon', 'Pennsylvania', 'Rhode Island', 'South Carolina',
       'South Dakota', 'Tennessee', 'Texas', 'Utah', 'Vermont',
       'Virginia', 'Washington']

confirmed = pd.read_csv(confirmed_fp)
ts_features = [item for item in confirmed.columns if item not in ['UID', 'iso2', 'iso3', 'code3', 'FIPS', 'Admin2', 'Province_State',
       'Country_Region', 'Lat', 'Long_', 'Combined_Key', 'Population', 'Province/State', 'Country/Region', 'Long']]
confirmed = confirmed.groupby('Province_State')[ts_features].sum()
confirmed = confirmed.stack().reset_index().rename({'Province_State':'region','level_1':'target_end_date',0:'label'},axis=1)
confirmed['target_end_date'] = pd.to_datetime(confirmed['target_end_date'])
#confirmed = confirmed[confirmed['target_end_date'].isin(epiweek_end_dates)]
inc_cases = confirmed.loc[confirmed['target_end_date'].isin(epiweek_end_dates), 'label'].values - confirmed.loc[confirmed['target_end_date'].isin(epiweek_start_dates), 'label'].values
confirmed = confirmed[confirmed['target_end_date'].isin(epiweek_end_dates)]
confirmed['label'] = inc_cases


confirmed_score_results = dict()
confirmed_score_tmp = dict()
for k,v in  model_results.items():
    #if k in ['COVIDhub-baseline','UMass-MechBayes']:
    #    continue
    v = v.drop_duplicates(['region','target_end_date'],keep='first')
    tmp = pd.merge(confirmed,v,on=['region','target_end_date'],how='inner')
    dates = tmp['forecast_date'].unique()
    region_size = tmp['region'].unique().shape[0]
    confirmed_score_tmp[k] = tmp
    tmp = tmp[tmp['region'].isin(regions)]
    if len(tmp['region'].unique()) > 30:
        score = (tmp['label'] - tmp['value']).map(abs).mean()
        confirmed_score_results[k] = score
    else:
        print(k)


UChicago-CovidIL
YYG-ParamSearch
SWC-TerminusCM
STH-3PU
UChicago-CovidIL_100
CovidActNow-SEIR_CAN
UChicago-CovidIL_60
UChicago-CovidIL_10_+
Google_Harvard-CPF
RPI_UW-Mob_Collision
GT_CHHS-COVID19
UChicago-CovidIL_30_+
IHME-CurveFit
NotreDame-FRED
MITCovAlliance-SIR
UChicago-CovidIL_80
USACE-ERDC_SEIR
Auquan-SEIR
UCSB-ACTS
UChicago-CovidIL_40
JHU_UNC_GAS-StatMechPool


In [175]:
pd.Series(confirmed_score_results).sort_values()

GT-DeepCOVID                    10219.673616
Covid19Sim-Simulator            10261.091418
CU-select                       10269.851064
CU-scenario_low                 10270.085106
BPagano-RtDriven                10274.711170
Karlen-pypm                     10275.172340
USC-SI_kJalpha                  10282.063830
CU-scenario_high                10282.170213
CU-nochange                     10282.276596
CU-scenario_mid                 10282.680851
JCB-PRM                         10283.486209
SteveMcConnell-CovidComplete    10284.404255
LANL-GrowthRate                 10286.680851
OliverWyman-Navigator           10287.533750
Geneva-DetGrowth                10288.408051
UMass-MechBayes                 10291.553191
MSRA-DeepST                     10291.667499
COVIDhub-ensemble               10292.702128
LNQ-ens1                        10292.733743
JHUAPL-Bucky                    10293.607667
UA-EpiCovDA                     10293.872340
UCSD_NEU-DeepGLEAM              10294.835454
MOBS-GLEAM

In [ ]:
results = pd.DataFrame({'Model':['MSRA-DeepST','LNQ-ens1','UMass-MechBayes','CDC-ensemble','JCB-PRM'],
                        '2020-10-11':[3,1,2,5,6],
                         '2020-10-04':[2,3,8,4,15],
                         '2020-09-27':[5,4,8,2,3],
                         '2020-09-20':[1,4,8,6,7],
                         '2020-10-18':[2,9,3,10,15],
                         '2020-10-25':[1,2,3,5,12]})